## Getting the artist's exhibitions (announcements related to him)

In [1]:
artist = "A.K. Burns"
url = f"https://www.e-flux.com/announcements/?c[]=Contemporary%20Art&p[]={artist}" #Could replace %20 with a space

In [14]:
import html
import requests
from bs4 import BeautifulSoup

def get_html_of_artist(artist, contemporary = False):
    if contemporary:
        url = f"https://www.e-flux.com/announcements/?c[]=Contemporary%20Art&p[]={artist}"
    else:
        url = f"https://www.e-flux.com/announcements/?p[]={artist}"

    response = requests.get(url)
    html = response.text
    return html

In [91]:
artist = "Hans Ulrich Obrist"
html = get_html_of_artist(artist)
soup = BeautifulSoup(html, 'html.parser')

The HTML we get back:

In [92]:
soup

<!DOCTYPE html>

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, minimum-scale=1" name="viewport"/>
<title>Announcements - e-flux</title>
<meta content="en" name="DC.LANGUAGE"/>
<link href="/styles/cbplayer.css?v=20220809025938" media="all" rel="stylesheet" type="text/css"/>
<link href="/styles/cblightbox.css?v=20210802045344" media="all" rel="stylesheet" type="text/css"/>
<link href="/styles/daterangepicker.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/styles/main.css?v=20240226171903" media="all" rel="stylesheet" type="text/css"/>
<meta content="article" property="og:type"/>
<meta content="Announcements - e-flux" property="og:title"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="Announcements - e-flux" name="twitter:title"

--------------------------------------------------------------------------------------------

- We need to find the exhibitions from the code.<br>
Luckily, they seem to be stored all in the same format:

Example of what we're looking for in the HTML code:

```HTML

<a class="preview-announcement__title" href="/announcements/165278/lia-gangitano-to-receive-the-2018-audrey-irmas-award-for-curatorial-excellence/">
Lia Gangitano to receive the 2018 Audrey Irmas Award for Curatorial Excellence
</a>


or:

<a class="preview-announcement__title" href="/announcements/528799/rirkrit-tiravanijawe-don-t-recognise-what-we-don-t-see/">
Rirkrit Tiravanija<br/><em>We Don’t Recognise What We Don’t See</em>
</a>
```

### **Get announcements: easy case**

In [86]:
import numpy as np

def get_announcements_of_artist(artist, contemporary = False):
    html = get_html_of_artist(artist, contemporary)
    soup = BeautifulSoup(html, "html.parser")
    announcements = soup.find_all("a", class_="preview-announcement__title") #All announcements are in <a> tags with this class, see website
    return announcements

def process_announcements(announcements):
    announcements_list = []
    for announcement in announcements:
        announcement_dict = {}
        announcement_dict['id'] = announcement["href"].split("/")[2] #the link is in the form of /announcements/123456/linktext.../
        announcement_dict['link'] = announcement["href"]
    
        try:
            announcement_dict['id']= int(announcement_dict['id'])
        except:
            print(f"'ID' problem with announcement: {announcement}, id: {announcement_dict['id']}")

        #TODO announcement_dict['artist_name'] = TODO (after <a> tag, before </br> tag, but may not have a name)
            
        announcement_dict['title'] = None
        try:
            announcement_dict['title'] = announcement.find("em").text.strip()
        except:
            try:
                announcement_dict['title'] = announcement.text.strip()
            except:
                print(f"Problem with announcement: {announcement}")
        
        announcements_list.append(announcement_dict)

    return announcements_list

In [87]:
announcements = get_announcements_of_artist("Hans Ulrich Obrist")
process_announcements(announcements)

[{'id': 507604,
  'link': '/announcements/507604/sin-wai-kindreaming-the-end/',
  'title': 'Dreaming the End'},
 {'id': 531326,
  'link': '/announcements/531326/suzanne-valadona-world-of-her-own/',
  'title': 'A World of Her Own'},
 {'id': 528799,
  'link': '/announcements/528799/rirkrit-tiravanijawe-don-t-recognise-what-we-don-t-see/',
  'title': 'We Don’t Recognise What We Don’t See'},
 {'id': 527878,
  'link': '/announcements/527878/young-climate-prize-2023-winners/',
  'title': 'Young Climate Prize 2023 winners'},
 {'id': 513946,
  'link': '/announcements/513946/50th-anniversary/',
  'title': '50th anniversary'},
 {'id': 508921,
  'link': '/announcements/508921/hoffnung-hoffnung/',
  'title': 'HOFFNUNG? HOFFNUNG!'},
 {'id': 505131,
  'link': '/announcements/505131/celebrating-15-years/',
  'title': 'Celebrating 15 years'},
 {'id': 506587,
  'link': '/announcements/506587/exhibition-program-spring-summer-2023/',
  'title': 'Exhibition program spring/summer 2023'},
 {'id': 502068,
  

<details><summary><u>Special cases that we need to consider:</u></summary>

```Python
t = """
<a class="preview-announcement__title" href="/announcements/528799/rirkrit-tiravanijawe-don-t-recognise-what-we-don-t-see/">
Rirkrit Tiravanija<br/><em>We Don’t Recognise What We Don’t See</em>
</a>
"""
soup = BeautifulSoup(t, "html.parser")

# Find the 'em' tag
em_tag = soup.find('em')
# If the 'em' tag exists, extract its text
name = em_tag.get_text() if em_tag else None
```

### **Get announcements: hard case**

The problem is we can only gather maximum 30 exhibitions per artist this way, because the website only shows 30 exhibitions per artist if we don't scroll. We need to use something like Selenium to scroll the page and get all the exhibitions.



Here is what GPT-4 suggested, works for Picasso:


In [30]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

# Create a new instance of the Firefox driver
driver = webdriver.Firefox()

# Go to your page url
url_Picasso = "https://www.e-flux.com/announcements/?p[]=Pablo%20Picasso"
driver.get(url_Picasso)

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Now you can parse the page source with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Remember to close the driver
driver.quit()

In [31]:
len(soup.find_all("a", class_="preview-announcement__title"))

176

Works with Picasso, we got 176 out of 176 exhibitions

<details><summary><u> Case when we want to click "Load more" button (might need fixes)</u></summary>

```python

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Create a new instance of the Firefox driver
driver = webdriver.Firefox()

# Go to your page url
url_Picasso = "https://www.e-flux.com/announcements/?p[]=Pablo%20Picasso"
driver.get(url_Picasso)

# Wait for the "Load more" button to become clickable
wait = WebDriverWait(driver, 10)

while True:
    try:
        # Wait until the "Load more" button is clickable, and then click it
        load_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Load more"]')))
        load_more_button.click()

        # Wait for the page to load
        time.sleep(2)
    except Exception as e:
        # If an error occurs, that probably means there's no more "Load more" button
        break

# Now you can parse the page source with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Remember to close the driver
driver.quit()

```

## Collect exhibitions of all artists



Maybe we collect the data for those artists, which are among the contemporary ones, but we don't just collect the contemporary exhibitions by them, but all of their exhibitions? Seems reasonable

In [ ]:
#TODO